With this code I have tried to implement Logistic Regression using a shallow neural network to demonstrate forward and backward propagation. This is an open source UCI dataset for classifying emails as spam or not spam. This is just for understanding purposes about how gradients are calculated in a neural network and how we can use sigmoid activation function to compute paramters and minimize costs.

This also demostrates use of matrix multiplication for faster computation in a neural network using NumPy instead of using for loops that are about 500 times slower.

I have tried to implement my learning from DeepLearning Specialization course on Coursera on an open sourse data set

## Import required libraries 

### Logistic regression can be easily implemented using Scikit-Learn's functions but this one is just for learning purposes

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

### Load dataset into pandas and convert it to NumPy arrays for computation 

In [2]:
spam = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data', header  = None)
X = spam.loc[:,0:56]
y = spam.loc[:,57]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, stratify = y, random_state = 123)

# Converting to NumPy
X_train = X_train.as_matrix()
y_train = y_train.as_matrix()
X_test = X_test.as_matrix()
y_test = y_test.as_matrix()

In [3]:
num_train = X_train.shape[0]
num_test = X_test.shape[0]
num_var = X_train.shape[1]
train_vector = X_train.T
test_vector = X_test.T

print ("Number of training samples: " + str(num_train))
print ("Number of testing samples: " + str(num_test))
print ("Number of variables: " + str(num_var))

Number of training samples: 3680
Number of testing samples: 921
Number of variables: 57


### Defining required functions

In [4]:
def initialize_vectors(dim):
    w = np.zeros(dim).reshape(dim,1)
    b = 0    
    return w, b

In [5]:
def sigmoid(z):
    return (1/(1+np.exp(-z)))

In [6]:
def propagation(w, b, X, Y):
    m = X.shape[1]
    
    A = sigmoid(np.dot(w.T,X) + b)
    dw = (1/m)*np.dot(X,(A-Y).T)
    db = (1/m)*np.sum(A-Y)
    
    cost = (-1/m)*np.sum(np.log(A)*Y+np.log(1-A)*(1-Y))    
    cost = np.squeeze(cost)
    gradient = {"dw": dw,
                "db": db}
    return gradient, cost

In [7]:
def parameter(w, b, X, Y, iterations, learning_rate):    
    costs = []
    grads = {}
    for i in range(iterations):
        gradient, cost = propagation(w, b, X, Y)
        dw = gradient["dw"]
        db = gradient["db"]
        w = w - learning_rate*dw
        b = b - learning_rate*db
        if i % 100 == 0:
            costs.append(cost)    
    parameters = {"w": w,
                  "b": b}
    
    gradients = {"dw": dw,
                 "db": db}
    
    return parameters, gradients, costs

In [8]:
def predictions(w, b, X):
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    A = sigmoid(np.dot(w.T,X) + b)
    
    # The cut-off here is 0.5; This can be changed as per business requirements
    for i in range(A.shape[1]):
        if A[0,i] <= 0.5:
            Y_prediction[0,i] = 0
        else:
            Y_prediction[0,i] = 1
        pass    
    return Y_prediction

In [9]:
def log_reg(X_train, y_train, X_test, y_test, iterations, learning_rate):

    w, b = initialize_vectors(X_train.shape[0])
    parameters, grads, costs = parameter(w, b, X_train, y_train, iterations, learning_rate)
    w = parameters["w"]
    b = parameters["b"]
    Y_prediction_test = predictions(w, b, X_test)
    Y_prediction_train = predictions(w, b, X_train)
    
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - y_test)) * 100))
    results = {"costs": costs,
               "Y_prediction_test": Y_prediction_test, 
               "Y_prediction_train" : Y_prediction_train, 
               "w" : w, 
               "b" : b,
               "learning_rate" : learning_rate,
               "iterations": iterations}
    
    return results

### Running Logisitc Regression 

In [10]:
results = log_reg(train_vector, y_train, test_vector, y_test, iterations = 5000, learning_rate = 0.005)

train accuracy: 81.71195652173913 %
test accuracy: 84.25624321389793 %


This is a pretty decent accuracy. However, test accuracy is more than train accuracy, which shows underfitting. The code however demonstrates calculation of gradients, parameters and costs like a neural network would compute.
I will also upload a notebook showing Logisitic Regression using Scikit-Learn